In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructType, StructField, TimestampType, IntegerType
from pyspark.sql.functions import from_json, col

# Define the path to the jars on the EC2 instance
spark_jars_path = "/home/ec2-user/stream-processing-template/jars"  # <-- Update this path

spark = SparkSession.builder.appName("retail_pysaprk_consumer") \
    .config("spark.jars", f"{spark_jars_path}/commons-pool2-2.11.1.jar,"
            f"{spark_jars_path}/spark-sql-kafka-0-10_2.12-3.4.0.jar,"
            f"{spark_jars_path}/spark-streaming-kafka-0-10-assembly_2.12-3.4.0.jar") \
    .getOrCreate()


In [ ]:
# Define the schema for our data
schema = StructType([
    StructField("store_location", StringType(), True),
    StructField("time_of_purchase", TimestampType(), True),
    StructField("product_ID", StringType(), True),
    StructField("transaction_amount", IntegerType(), True)
])

# Stream from Kafka topic
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "b-1.monstercluster1.6xql65.c3.kafka.eu-west-2.amazonaws.com:9092") \
    .option("subscribe", "retail_transactions") \
    .load()

In [ ]:
transactions = (df.selectExpr("CAST(value AS STRING)")
                .withColumn("data", from_json(col("value"), schema))
                .select("data.*"))

query = transactions.writeStream \
        .outputMode("append") \
        .format("memory") \
        .queryName("transanctions_stream") \
        .start()

query.awaitTermination(10)

In [ ]:
processed_data = spark.sql("SELECT * FROM transanctions_stream")

In [ ]:
from pyspark.sql.functions import sum
total_transactions = processed_data.groupBy("store_location").agg(sum("transaction_amount").alias("total_amount"))
total_transactions.show()

In [ ]:
transaction_counts = processed_data.groupBy("store_location").count().withColumnRenamed("count", "transaction_count")
transaction_counts.show()

In [ ]:
from pyspark.sql.functions import avg
average_transactions = processed_data.groupBy("store_location").agg(avg("transaction_amount").alias("average_amount"))
average_transactions.show()

In [ ]:
from pyspark.sql.functions import col
discounted_amount = processed_data.withColumn("discounted_amount", col("transaction_amount") * 0.9)
discounted_amount.orderBy(discounted_amount.time_of_purchase.desc()).show()